In [ ]:
import main
import importlib
importlib.reload(main)

import logging
import os
import pandas
import nltk
import time
import numpy as np
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torchtext.vocab as torchvocab

In [ ]:
dm = main.DataManager()

train_set = main.DataManager.load_data(os.path.join(main.Constants.Data.datadir, main.Constants.Data.train))
train_set, val_set = main.DataManager.train_val_split(train_set)
test_set = main.DataManager.load_data(os.path.join(main.Constants.Data.datadir, main.Constants.Data.test))

train_set = train_set[:1000]
val_set = val_set[:1000]
test_set = test_set[:1000]

train_set["token_ids"] = dm.sentences_to_indices(train_set.question_text)
val_set["token_ids"] = dm.sentences_to_indices(val_set.question_text)
test_set["token_ids"] = dm.sentences_to_indices(test_set.question_text)

In [ ]:
model = main.LstmModel(dm.embedding_layer)

In [ ]:
class Trainer() :
    def __init__(self, model) :
        self.model = model
        self.optimizer = torch.optim.Adadelta([p for p in model.parameters() if p.requires_grad])

    def train(self, train_set) :
        for eps in range(25) :
            logging.info("Epoch : {}".format(eps))

            for batch in main.DataManager.batch(train_set) :
                self.optimizer.zero_grad()
                loss = self.model.step(main.DataManager.batch_indices_to_tensor(batch.token_ids), torch.from_numpy(batch.target.as_matrix()))
                logging.info(loss)
                self.optimizer.step()
            
            print(model.calculate_f1(main.DataManager.batch_indices_to_tensor(train_set.token_ids), torch.from_numpy(train_set.target.as_matrix())))
        return model.predict(main.DataManager.batch_indices_to_tensor(train_set.token_ids))

In [ ]:
trainer = Trainer(model)
pred = trainer.train(train_set)